# Dubai — Quantum accurate bond inference and partial charge calculations

See [the tutorial](/Tutorials/ligand_bond_inference_and_partial_charge_calculation.ipynb) for a detailed explanation.

In [ ]:
# |hide
import os
import pathlib

WORK_DIR = pathlib.Path("~/qdx/dubai_quickstart/").expanduser()
if WORK_DIR.exists():
    !rm -r $WORK_DIR
os.makedirs(WORK_DIR)
os.chdir(WORK_DIR)
YOUR_TOKEN = os.getenv("RUSH_TOKEN")

In [ ]:
import json
from pathlib import Path

import requests

import rush

os.environ["RUSH_TOKEN"] = YOUR_TOKEN

client = rush.build_blocking_provider_with_functions(
    batch_tags=["bond_inference_quickstart"]
)

# Convert aspirin to a QDXF file so we can use it for this demo
SMILES_STRING = "CC(=O)OC1=CC=CC=C1C(=O)O"
SDF_LINK = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/smiles/{SMILES_STRING}/record/SDF?record_type=3d"
aspirin_sdf_file = Path(client.workspace / "aspirin.sdf")
aspirin_sdf_file.write_bytes(requests.get(SDF_LINK).content)
(ligand_qdxf,) = client.convert("SDF", aspirin_sdf_file)

ligand_dict = json.loads(ligand_qdxf.download().read_text())

# Remove connectivity
EXPECTED_CONNECTIVITY = ligand_dict[0]["topology"]["connectivity"]
ligand = ligand_dict[0]
del ligand["topology"]["connectivity"]
ligand["topology"]["fragment_multiplicities"] = [1]

LIGAND_FILEPATH = client.workspace / "aspirin.qdxf.json"
LIGAND_FILEPATH.write_text(json.dumps(ligand))

# Run the inference
(ligand_with_bonds,) = client.dubai(
    LIGAND_FILEPATH, resources={"gpus": 1, "storage": 1024_000, "walltime": 60}
)

output_ligand = json.loads(ligand_with_bonds.download().read_text())

for expected_bond, inferred_bond in zip(
    EXPECTED_CONNECTIVITY, output_ligand["topology"]["connectivity"]
):
    # Check start atoms are the same
    assert expected_bond[0] == inferred_bond[0]
    # Check ending atoms are the same
    assert expected_bond[1] == inferred_bond[1]
    # NB: we don't check the third item of the bond -- the bond type. This is
    # because Dubai accurately outputs ring bonds as a specific 'RINGBOND' type,
    # whereas SDF aspirin was interleaving single and double bonds.
print("Bond inference performed correctly!")

2024-04-10 16:01:28,488 - rush - INFO - Not restoring by default via default
2024-04-10 16:01:31,737 - rush - INFO - Argument b171f51b-0ae1-461f-8afb-46528ee59595 is now ModuleInstanceStatus.RESOLVING
2024-04-10 16:01:35,053 - rush - INFO - Argument b171f51b-0ae1-461f-8afb-46528ee59595 is now ModuleInstanceStatus.ADMITTED
2024-04-10 16:01:38,691 - rush - INFO - Argument b171f51b-0ae1-461f-8afb-46528ee59595 is now ModuleInstanceStatus.AWAITING_UPLOAD
2024-04-10 16:01:55,349 - rush - INFO - Argument 09196a92-09e7-4e81-bf29-190210598bd8 is now ModuleInstanceStatus.RESOLVING
2024-04-10 16:01:58,631 - rush - INFO - Argument 09196a92-09e7-4e81-bf29-190210598bd8 is now ModuleInstanceStatus.ADMITTED
2024-04-10 16:05:15,767 - rush - INFO - Argument 09196a92-09e7-4e81-bf29-190210598bd8 is now ModuleInstanceStatus.RUNNING
2024-04-10 16:05:22,552 - rush - INFO - Argument 09196a92-09e7-4e81-bf29-190210598bd8 is now ModuleInstanceStatus.AWAITING_UPLOAD
Bond inference performed correctly!
